In [1]:
# Importing the necessary modules
import os 
import cv2 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Setting the path to the image 
banditsImage = "bandits.jpg"

# Setting the path to the txt file 
yoloFilePath = "bandits.txt"

# loading the image into memory
img = cv2.imread(banditsImage) 

In [5]:
# --- 1. File Paths and Image Dimensions ---
YOLO_FILE_PATH = "bandits.txt"
IMAGE_PATH = "bandits.jpg"



# --- 2. Load the Image and Setup Constants ---
image = cv2.imread(IMAGE_PATH)

IMAGE_HEIGHT = 500 
IMAGE_WIDTH = 500

# if image is None:
#     print(f"Error: Could not load image at {IMAGE_PATH}. Check the file path.")
#     # Fallback: Create a blank image if the file is missing for demonstration
#     image = np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, 3), dtype=np.uint8)
#     print("Using a blank image for demonstration.")
    
box_color = (0, 255, 0) # Green (BGR format)
box_thickness = 2
label_map = {0: "bandits"} # Map class ID '0' to the object name

def yolo_to_pixel(yolo_coords_line, img_w, img_h):
    """Converts a line of normalized YOLO coordinates to absolute pixel coordinates."""
    
    # Split the line into its 5 components and convert them to float/int
    parts = yolo_coords_line.split()
    if len(parts) != 5:
        print(f"Skipping malformed line: {yolo_coords_line}")
        return None, None, None, None, None

    class_id = int(parts[0])
    x_center_norm, y_center_norm, width_norm, height_norm = map(float, parts[1:])
    
    # Conversion Formulas:
    abs_width = width_norm * img_w
    abs_height = height_norm * img_h
    abs_center_x = x_center_norm * img_w
    abs_center_y = y_center_norm * img_h
    
    xmin = int(abs_center_x - (abs_width / 2))
    ymin = int(abs_center_y - (abs_height / 2))
    xmax = int(abs_center_x + (abs_width / 2))
    ymax = int(abs_center_y + (abs_height / 2))
    
    return xmin, ymin, xmax, ymax, class_id

# --- 3. Load Data from .txt File and Process ---

try:
    # Use 'with open' to ensure the file is properly closed
    with open(YOLO_FILE_PATH, 'r') as file:
        yolo_data_lines = file.readlines()
        
except FileNotFoundError:
    print(f"Error: YOLO data file not found at {YOLO_FILE_PATH}")
    yolo_data_lines = [] # Set to empty list to skip drawing

if yolo_data_lines:
    print(f"Successfully loaded {len(yolo_data_lines)} lines from {YOLO_FILE_PATH}.")
    
    for line in yolo_data_lines:
        # Clean the line (remove leading/trailing whitespace, like the newline character)
        line = line.strip()
        if not line:
            continue # Skip empty lines

        xmin, ymin, xmax, ymax, class_id = yolo_to_pixel(line, IMAGE_WIDTH, IMAGE_HEIGHT)
        
        if xmin is not None:
            object_name = label_map.get(class_id, f"Class {class_id}")
            
            # Draw Bounding Box
            start_point = (xmin, ymin)
            end_point = (xmax, ymax)
            
            cv2.rectangle(image, start_point, end_point, box_color, box_thickness)
            
            # Optional: Add label
            cv2.putText(
                image, 
                object_name, 
                (xmin, ymin - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.5, 
                box_color, 
                2 
            )
            
            print(f"Drew box for '{object_name}' with pixel coordinates: ({xmin}, {ymin}) to ({xmax}, {ymax})")
else:
    print("No bounding box data to process.")

# --- 4. Save the Result ---
output_filename = "result_yolo_boxes_from_file.jpg"
cv2.imwrite(output_filename, image)
print(f"\nSuccessfully saved image with boxes to: {output_filename}")

Successfully loaded 2 lines from bandits.txt.
Drew box for 'bandits' with pixel coordinates: (60, 7) to (212, 464)
Drew box for 'bandits' with pixel coordinates: (237, 27) to (372, 479)

Successfully saved image with boxes to: result_yolo_boxes_from_file.jpg


In [ ]:
image = cv2.imread("result_yolo_boxes_from_file.jpg") 
cv2.resize(image, (400, 400)) 

cv2.imshow("Analyzed Image", image) 
cv2.waitKey(0) 
cv2.destroyAllWindows() 

In [3]:
# Using xml data 
import xml.etree.ElementTree as ET
import cv2
import numpy as np
import os

# --- 1. XML File Path ---
xml_file_path = "bandits.xml" # This is the path to your XML annotation file

def extract_and_draw(file_path):
    """
    Parses XML data from a file, reads the image, and draws bounding boxes.
    """
    try:
        # 2. Extract Data from XML File
        # Use ET.parse() to read and parse the XML file
        tree = ET.parse(file_path)
        root = tree.getroot()
        
    except FileNotFoundError:
        print(f"Error: XML file not found at {file_path}")
        return
    except ET.ParseError as e:
        print(f"Error parsing XML file: {e}")
        return
    
    # Get the image file path
    image_path_element = root.find('path')
    if image_path_element is None or not image_path_element.text:
        # If 'path' is missing or empty, try combining 'folder' and 'filename'
        folder = root.find('folder').text if root.find('folder') is not None else ''
        filename = root.find('filename').text if root.find('filename') is not None else ''
        image_path = os.path.join(folder, filename)
    else:
        image_path = image_path_element.text
        
    print(f"Attempting to load image from: {image_path}")

    # Load the image using OpenCV
    image = cv2.imread(image_path)
    
    if image is None:
        print(f"Error: Could not load image at {image_path}. Check the file path.")
        return

    # Define color and thickness for the bounding boxes (BGR format)
    box_color = (0, 255, 0)  # Green
    box_thickness = 2
    
    # Iterate through all 'object' tags
    for obj in root.findall('object'):
        object_name = obj.find('name').text
        bndbox = obj.find('bndbox')
        
        if bndbox is None:
            print(f"Warning: Missing bounding box for object '{object_name}'")
            continue

        # Get coordinates, convert to integers
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        # 3. Draw Bounding Box on Image
        start_point = (xmin, ymin)
        end_point = (xmax, ymax)
        
        cv2.rectangle(image, start_point, end_point, box_color, box_thickness)
        
        # Optional: Add the object name label above the box
        cv2.putText(
            image, 
            object_name, 
            (xmin, ymin - 10), # Position text slightly above the box
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.5, # Font size
            box_color, 
            2 # Text thickness
        )
        
        print(f"Drew box for '{object_name}': ({xmin}, {ymin}) to ({xmax}, {ymax})")

    # 4. Display or Save the Result
    output_filename = "result_" + root.find('filename').text
    cv2.imwrite(output_filename, image)
    print(f"\nSuccessfully saved image with boxes to: {output_filename}")
    
# Run the function with the file path
extract_and_draw(xml_file_path)

Attempting to load image from: /Users/mbonuchinedu/Documents/Projects/banditsProject/bandits.jpg
Drew box for 'bandits': (156, 11) to (543, 669)
Drew box for 'bandits': (607, 40) to (954, 691)

Successfully saved image with boxes to: result_bandits.jpg


In [4]:
image = cv2.imread("result_bandits.jpg") 
cv2.resize(image, (400, 400)) 

cv2.imshow("Analyzed Image", image) 
cv2.waitKey(0) 
cv2.destroyAllWindows() 